**自然语言处理**是指研究使用自然语言的计算机和人类之间的交互。在实践中，使用自然语言处理技术来处理和分析文本数据是非常常见的。要理解文本，我们可以从学习它的表示开始。利用来自大型语料库的现有文本序列，**自监督学习**（self-supervised learning）已被广泛用于预训练文本表示，例如通过使用周围文本的其它部分来预测文本的隐藏部分。通过这种方式，模型可以通过有监督地从**海量**文本数据中学习，而不需要**昂贵**的标签标注！

本章我们将看到：当将每个单词或子词视为单个词元时，可以在大型语料库上使用word2vec、GloVe或子词嵌入模型预先训练每个词元的词元。**经过预训练后，每个词元的表示可以是一个向量。但是，无论上下文是什么，它都保持不变**。例如，“bank”（可以译作银行或者河岸）的向量表示在“go to the bank to deposit some money”（去银行存点钱）和“go to the bank to sit down”（去河岸坐下来）中是相同的。因此，许多较新的预训练模型使相同词元的表示适应于不同的上下文，其中包括基于transformer编码器的更深的自监督模型BERT。在本章中，我们将重点讨论如何预训练文本的这种表示，
如下图中所强调的那样。

![预训练好的文本表示可以放入各种深度学习架构，应用于不同自然语言处理任务（本章主要研究上游文本的预训练）](../img/14_1.png)

显示了预训练好的文本表示可以放入各种深度学习架构，应用于不同自然语言处理任务。

# 14.1. 词嵌入（Word2vec）
自然语言是用来表达人脑思维的复杂系统。在这个系统中，词是意义的基本单元。顾名思义，**词向量**是用于表示单词意义的向量，并且还可以被认为是单词的特征向量或表示。将单词映射到实向量的技术称为**词嵌入**。近年来，词嵌入逐渐成为自然语言处理的基础知识。

## 14.1.1. 为何独热向量是一个糟糕的选择
虽然独热向量很容易构建，但它们通常不是一个好的选择。一个主要原因是独热向量不能准确表达不同词之间的相似度，比如我们经常使用的“余弦相似度”。对于向量$\mathbf{x}, \mathbf{y} \in \mathbb{R}^d$，它们的余弦相似度是它们之间角度的余弦：

$$\frac{\mathbf{x}^\top \mathbf{y}}{\|\mathbf{x}\| \|\mathbf{y}\|} \in [-1, 1]. \tag{14.1.1}$$

由于任意两个不同词的独热向量之间的余弦相似度为0，所以独热向量不能编码词之间的相似性。

## 14.1.2. 自监督的word2vec
[word2vec](https://code.google.com/archive/p/word2vec/)工具是为了解决上述问题而提出的。它将每个词映射到一个固定长度的向量，这些向量能更好地表达不同词之间的相似性和类比关系。word2vec工具包含两个模型，即**跳元模型**（skip-gram）和**连续词袋**（CBOW）。对于在语义上有意义的表示，它们的训练依赖于条件概率，条件概率可以被看作是**使用语料库中一些词来预测另一些单词**。由于是不带标签的数据，因此跳元模型和连续词袋都是自监督模型。

下面，我们将介绍这两种模式及其训练方法。

## 14.1.3. 跳元模型（Skip-Gram）
跳元模型假设一个词可以用来在文本序列中生成其周围的单词。以文本序列“the”、“man”、“loves”、“his”、“son”为例。假设**中心词**选择“loves”，并将上下文窗口设置为2，如图所示，给定中心词“loves”，跳元模型考虑生成**上下文词**“the”、“man”、“him”、“son”的条件概率：

$$P(\textrm{"the"},\textrm{"man"},\textrm{"his"},\textrm{"son"}\mid\textrm{"loves"}). \tag{14.1.2}$$

假设上下文词是在给定中心词的情况下独立生成的（即条件独立性）。在这种情况下，上述条件概率可以重写为：

$$P(\textrm{"the"}\mid\textrm{"loves"})\cdot P(\textrm{"man"}\mid\textrm{"loves"})\cdot P(\textrm{"his"}\mid\textrm{"loves"})\cdot P(\textrm{"son"}\mid\textrm{"loves"}). \tag{14.1.3}$$

![跳元模型考虑了在给定中心词的情况下生成周围上下文词的条件概率](../img/14_2.png)

在跳元模型中，每个词都有两个$d$维向量表示，用于计算条件概率。更具体地说，对于词典中索引为$i$的任何词，分别用$\mathbf{v}_i\in\mathbb{R}^d$和$\mathbf{u}_i\in\mathbb{R}^d$表示其用作**中心词**和**上下文词**时的两个向量。给定中心词$w_c$（词典中的索引$c$），生成任何上下文词$w_o$（词典中的索引$o$）的条件概率可以通过对向量点积的softmax操作来建模：

$$P(w_o \mid w_c) = \frac{\text{exp}(\mathbf{u}_o^\top \mathbf{v}_c)}{ \sum_{i \in \mathcal{V}} \text{exp}(\mathbf{u}_i^\top \mathbf{v}_c)}, \tag{14.1.4}$$

其中词表索引集$\mathcal{V} = \{0, 1, \ldots, |\mathcal{V}|-1\}$。给定长度为$T$的文本序列，其中时间步$t$处的词表示为$w^{(t)}$。假设上下文词是在给定任何中心词的情况下独立生成的。对于上下文窗口$m$，跳元模型的似然函数是在给定任何中心词的情况下生成所有上下文词的概率：

$$ \prod_{t=1}^{T} \prod_{-m \leq j \leq m,\ j \neq 0} P(w^{(t+j)} \mid w^{(t)}),\tag{14.1.5}$$

其中可以省略小于$1$或大于$T$的任何时间步。

### 训练

跳元模型参数是词表中每个词的中心词向量和上下文词向量。在训练中，我们通过最大化似然函数（即极大似然估计）来学习模型参数。这相当于最小化以下损失函数：

$$ - \sum_{t=1}^{T} \sum_{-m \leq j \leq m,\ j \neq 0} \text{log}\, P(w^{(t+j)} \mid w^{(t)}).\tag{14.1.6}$$

当使用随机梯度下降来最小化损失时，在每次迭代中可以随机抽样一个较短的子序列来计算该子序列的（随机）梯度，以更新模型参数。为了计算该（随机）梯度，我们需要获得对数条件概率关于中心词向量和上下文词向量的梯度。通常，根据公式14.1.4，涉及中心词$w_c$和上下文词$w_o$的对数条件概率为：

$$\log P(w_o \mid w_c) =\mathbf{u}_o^\top \mathbf{v}_c - \log\left(\sum_{i \in \mathcal{V}} \text{exp}(\mathbf{u}_i^\top \mathbf{v}_c)\right).\tag{14.1.7}$$

通过微分，我们可以获得其相对于中心词向量$\mathbf{v}_c$的梯度为

$$\begin{aligned}\frac{\partial \text{log}\, P(w_o \mid w_c)}{\partial \mathbf{v}_c}&= \mathbf{u}_o - \frac{\sum_{j \in \mathcal{V}} \exp(\mathbf{u}_j^\top \mathbf{v}_c)\mathbf{u}_j}{\sum_{i \in \mathcal{V}} \exp(\mathbf{u}_i^\top \mathbf{v}_c)}\\&= \mathbf{u}_o - \sum_{j \in \mathcal{V}} \left(\frac{\text{exp}(\mathbf{u}_j^\top \mathbf{v}_c)}{ \sum_{i \in \mathcal{V}} \text{exp}(\mathbf{u}_i^\top \mathbf{v}_c)}\right) \mathbf{u}_j\\&= \mathbf{u}_o - \sum_{j \in \mathcal{V}} P(w_j \mid w_c) \mathbf{u}_j.\end{aligned} \tag{14.1.8}$$

注意，公式14.1.8中的计算需要词典中以$w_c$为中心词的所有词的条件概率。其他词向量的梯度可以以相同的方式获得。

对词典中索引为$i$的词进行训练后，得到$\mathbf{v}_i$（作为中心词）和$\mathbf{u}_i$（作为上下文词）两个词向量。在自然语言处理应用中，跳元模型的中心词向量通常用作词表示。




## 14.1.4. 连续词袋（CBOW）模型
**连续词袋**（CBOW）模型类似于跳元模型。与跳元模型的主要区别在于，连续词袋模型**假设中心词是基于其在文本序列中的周围上下文词生成的**。例如，在文本序列“the”、“man”、“loves”、“his”、“son”中，在“loves”为中心词且上下文窗口为2的情况下，连续词袋模型考虑基于上下文词“the”、“man”、“him”、“son”（如下图所示）生成中心词“loves”的条件概率，即：

$$P(\textrm{"loves"}\mid\textrm{"the"},\textrm{"man"},\textrm{"his"},\textrm{"son"}). \tag{14.1.9}$$

![连续词袋模型考虑了给定周围上下文词生成中心词条件概率](../img/14_3.png)


由于连续词袋模型中存在多个上下文词，因此在计算条件概率时对这些上下文词向量进行平均。具体地说，对于字典中索引$i$的任意词，分别用$\mathbf{v}_i\in\mathbb{R}^d$和$\mathbf{u}_i\in\mathbb{R}^d$表示用作**上下文**词和**中心**词的两个向量（符号与跳元模型中相反）。给定上下文词$w_{o_1}, \ldots, w_{o_{2m}}$（在词表中索引是$o_1, \ldots, o_{2m}$）生成任意中心词$w_c$（在词表中索引是$c$）的条件概率可以由以下公式建模:

$$P(w_c \mid w_{o_1}, \ldots, w_{o_{2m}}) = \frac{\text{exp}\left(\frac{1}{2m}\mathbf{u}_c^\top (\mathbf{v}_{o_1} + \ldots, + \mathbf{v}_{o_{2m}}) \right)}{ \sum_{i \in \mathcal{V}} \text{exp}\left(\frac{1}{2m}\mathbf{u}_i^\top (\mathbf{v}_{o_1} + \ldots, + \mathbf{v}_{o_{2m}}) \right)}. \tag{14.1.10}$$

为了简洁起见，我们设为$\mathcal{W}_o= \{w_{o_1}, \ldots, w_{o_{2m}}\}$和$\bar{\mathbf{v}}_o = \left(\mathbf{v}_{o_1} + \ldots, + \mathbf{v}_{o_{2m}} \right)/(2m)$。那么公式14.1.10可以简化为：

$$P(w_c \mid \mathcal{W}_o) = \frac{\exp\left(\mathbf{u}_c^\top \bar{\mathbf{v}}_o\right)}{\sum_{i \in \mathcal{V}} \exp\left(\mathbf{u}_i^\top \bar{\mathbf{v}}_o\right)}. \tag{14.1.11} $$

给定长度为$T$的文本序列，其中时间步$t$处的词表示为$w^{(t)}$。对于上下文窗口$m$，连续词袋模型的似然函数是在给定其上下文词的情况下生成所有中心词的概率：

$$ \prod_{t=1}^{T}  P(w^{(t)} \mid  w^{(t-m)}, \ldots, w^{(t-1)}, w^{(t+1)}, \ldots, w^{(t+m)}). \tag{14.1.12}$$

### 训练
训练连续词袋模型与训练跳元模型几乎是一样的。连续词袋模型的最大似然估计等价于最小化以下损失函数：

$$  -\sum_{t=1}^T  \text{log}\, P(w^{(t)} \mid  w^{(t-m)}, \ldots, w^{(t-1)}, w^{(t+1)}, \ldots, w^{(t+m)}). \tag{14.1.13}$$

请注意，

$$\log\,P(w_c \mid \mathcal{W}_o) = \mathbf{u}_c^\top \bar{\mathbf{v}}_o - \log\,\left(\sum_{i \in \mathcal{V}} \exp\left(\mathbf{u}_i^\top \bar{\mathbf{v}}_o\right)\right). \tag{14.1.14}$$

通过微分，我们可以获得其关于任意上下文词向量$\mathbf{v}_{o_i}$（$i = 1, \ldots, 2m$）的梯度，如下：

$$\frac{\partial \log\, P(w_c \mid \mathcal{W}_o)}{\partial \mathbf{v}_{o_i}} = \frac{1}{2m} \left(\mathbf{u}_c - \sum_{j \in \mathcal{V}} \frac{\exp(\mathbf{u}_j^\top \bar{\mathbf{v}}_o)\mathbf{u}_j}{ \sum_{i \in \mathcal{V}} \text{exp}(\mathbf{u}_i^\top \bar{\mathbf{v}}_o)} \right) = \frac{1}{2m}\left(\mathbf{u}_c - \sum_{j \in \mathcal{V}} P(w_j \mid \mathcal{W}_o) \mathbf{u}_j \right). \tag{14.1.15}$$

其他词向量的梯度可以以相同的方式获得。与跳元模型不同，连续词袋模型通常使用上下文词向量作为词表示。

## 14.1.5. 小结
* 词向量是用于表示单词意义的向量，也可以看作是词的特征向量。将词映射到实向量的技术称为词嵌入。
* word2vec工具包含跳元模型和连续词袋模型。
* 跳元模型假设一个单词可用于在文本序列中，生成其周围的单词；而连续词袋模型假设基于上下文词来生成中心单词。

# 一、Skip-gram 和 CBOW 是什么？

**Skip-gram 和 CBOW** 是两种经典的 **Word2Vec（词向量）训练方法**，由 Mikolov 等人于 2013 年提出，用于将词语编码成可用于神经网络处理的低维稠密向量（dense vector）。

这两种方法本质上都是使用**浅层神经网络**通过上下文信息来捕捉词语的语义关系，但预测的方向不同。

---

## 1. Skip-gram：**用中心词预测上下文**

### 目标：

给定中心词 $w_t$，预测其前后上下文词 $\{w_{t - c}, ..., w_{t + c} \}$

### 示例：

句子：“The cat sits on the mat”

假设窗口大小 $c = 2$，以“sits”为中心词：

模型尝试学习：

$$
P(\text{on}|\text{sits}),\quad P(\text{cat}|\text{sits})
$$

### 损失函数（简化）：

$$
\mathcal{L} = -\sum_{-c \leq j \leq c, j \neq 0} \log P(w_{t+j} | w_t)
$$

---

## 2. CBOW（Continuous Bag-of-Words）：**用上下文预测中心词**

### 目标：

给定上下文词 $\{w_{t - c}, ..., w_{t + c} \}$，预测中心词 $w_t$

### 示例：

仍然是“sits”为中心词：

模型尝试学习：

$$
P(\text{sits}|\text{The, cat, on, the})
$$

### 损失函数（简化）：

$$
\mathcal{L} = -\log P(w_t | w_{t-c}, ..., w_{t+c})
$$

---

# 二者的联系与区别

| 对比项     | Skip-gram              | CBOW      |
| ------- | ---------------------- | --------- |
| 预测方向    | 中心词 → 上下文              | 上下文 → 中心词 |
| 模型训练难度  | 慢但更准确                  | 快但精度略低    |
| 对低频词    | 表现好                    | 效果差       |
| 向量学习方式  | 每个词有 input/output 两个向量 | 同上        |
| 是否使用负采样 | 常用                     | 常用        |
| 表示风格    | 更细粒度，语义捕捉强             | 平滑、聚合式    |

>  **联系**：都是 Word2Vec 的两种训练策略，本质上属于**浅层语言建模方法（shallow embedding model）**，目标是学出好的 word embedding。

---

# 三、当前主流 LLM（大语言模型）使用的词表示方法

### 不再使用 Word2Vec，而是使用 **上下文相关的表示（contextualized embedding）**

> LLM（如 GPT、BERT 等）不再采用静态的词向量（如 Word2Vec），而是使用上下文相关的动态向量表示。

### 主要技术：

| 模型              | 表示方法                                                         | 特点                     |
| --------------- | ------------------------------------------------------------ | ---------------------- |
| BERT / GPT      | 使用 **Token Embedding + Positional Embedding**，输入 Transformer | 词向量随上下文变化              |
| WordPiece / BPE | 使用 **子词级别（Subword-level）词表**，如 “play”, “##ing”               | 避免词表过大，处理未登录词          |
| Embedding 表层    | 每个 token 映射为一个向量（如 768 维）                                    | 作为 transformer 输入的第一个层 |

### 优势：

* **动态词义建模**（“bank”在“river bank”和“money bank”中意义不同）
* **子词分解**，提高泛化能力
* **不再需要提前训练词向量**，模型直接在下游任务中 end-to-end 学习表示

---

# 总结笔记版

> * Skip-gram 和 CBOW 是 Word2Vec 提出的两种词向量训练方法，分别以“中心词预测上下文”与“上下文预测中心词”为目标；
> * Skip-gram 表现更好，尤其在稀有词场景中，但训练慢；
> * 现代大型语言模型（LLM）不再使用静态的 Word2Vec 表示，而是采用上下文相关的 **Transformer + 子词分解 + Embedding 层**，学习更灵活的 token 表示；
> * Word2Vec 的思想仍影响着 NLP embedding 模型的发展（如 node2vec、doc2vec、graph2vec）。


